### Este Notebook estara enfocado en el desanidado y normalizacion de los datos de la columna `MISC`. Al terminar continuar con el Notebook 4.

In [2]:
import pandas as pd

In [ ]:
# Leemos la tabla g_sitios que tenemos hasta el momento.
g_sitios = pd.read_parquet(r'C:\Users\mauri\OneDrive\Escritorio\proyectogrupal\data\google_sitios.parquet', engine='pyarrow')

In [ ]:
# Creamos una nueva tabla misc
misc_sitios = g_sitios[['gmap_id', 'MISC']]

# Donde MISC sea NULO no hay Información Se pueden Borrar ya que esta tabla es SOLO MISCELANEOS.
misc_sitios = misc_sitios.dropna(subset=['MISC'])

In [ ]:
# Se Define una Función que convierte los Diccionarios en Listas para poder Aplicar `explode` despues.  Sólo se toman aquellos pares del diccionario donde value is not None
def dict_to_list(diccionario:dict) -> list:
    """ Devuelve el Diccionario de la columna `MISC` y lo retorna
                    como una lista de String "key:Value" Siempre y cuando el 
                    value de la pareja no sea Nulo.

    Args:
        diccionario (dict): diccionario recibido

    Returns:
        list: Lista a devolver
    """
    return [f"{key}: {value}" for key, value in diccionario.items() if value is not None]

In [ ]:
# Aplicamos la funcion definida arriba.
misc_sitios['lista_misc'] = misc_sitios['MISC'].apply(dict_to_list)

# Muestra parcial de la tabla:
misc_sitios.head()

Se borra la columna MISC, se aplica explode al dataframe con respecto a la columna `lista_misc` y luego renombrar la columna para tener lista la tabla

In [ ]:
misc_sitios = misc_sitios.drop(columns=['MISC'])

misc_sitios = misc_sitios.explode('lista_misc')

misc_sitios.rename(columns={'lista_misc': 'misc'}, inplace=True)
misc_sitios.head()

In [ ]:
# Guaradamos la tabla desanidada que se conecta a travez de gmap_id con la tabla principal.
misc_sitios.to_parquet('datasets/google/sitios/google_misc_sitios.parquet')

--- 

### Volver a esta parte del notebook luego de ejecutar el notebook 7.g_alcance donde se quedaron las filas necesarias para proseguir con esta tabla

In [26]:
# Vemos la tabla con las filas actualizadas para actualizar tipos de datos
import pandas as pd
misc = pd.read_parquet(r'C:\Users\mauri\OneDrive\Escritorio\proyectogrupal\data\g_misc_v2.parquet', engine='pyarrow')

In [27]:
pd.set_option('display.max_colwidth', None)
misc

,id_negocio,misc
0,0x89c2f6163ac2864f:0xa4213a2f66c15d8f,Accessibility: ['Wheelchair accessible entrance']
1,0x864c1f10c079a921:0x95710cd26231cc3,Accessibility: ['Wheelchair accessible entrance']
2,0x8824d759c04cb37b:0xef1cb7398b3d692e,Accessibility: ['Wheelchair accessible entrance']
3,0x88d9050582e1e64b:0x33457ba08c64df20,Health & safety: ['Appointment required' 'Mask required' 'Temperature check required'\n 'Staff wear masks' 'Staff get temperature checks'\n 'Staff required to disinfect surfaces between visits']
4,0x88d9050582e1e64b:0x33457ba08c64df20,Service options: ['Online appointments']
...,...,...
82026,0x89b62c2db5856cd3:0x89e0d1a12b77f78a,Accessibility: ['Wheelchair accessible entrance']
82027,0x89b7fdc26ba070ef:0x376e99b8fe4bfc32,Accessibility: ['Wheelchair accessible entrance' 'Wheelchair accessible parking lot']
82028,0x54c0e365e43253f9:0x4366520651be08bb,Accessibility: ['Wheelchair accessible parking lot']
82029,0x54c0e365e43253f9:0x4366520651be08bb,Planning: ['Quick visit']


In [28]:
# Extraer el texto antes del primer ':'
misc['category'] = misc['misc'].str.split(':').str[0]

# Obtener las categorías únicas
unique_categories = misc['category'].unique()
misc

,id_negocio,misc,category
0,0x89c2f6163ac2864f:0xa4213a2f66c15d8f,Accessibility: ['Wheelchair accessible entrance'],Accessibility
1,0x864c1f10c079a921:0x95710cd26231cc3,Accessibility: ['Wheelchair accessible entrance'],Accessibility
2,0x8824d759c04cb37b:0xef1cb7398b3d692e,Accessibility: ['Wheelchair accessible entrance'],Accessibility
3,0x88d9050582e1e64b:0x33457ba08c64df20,Health & safety: ['Appointment required' 'Mask required' 'Temperature check required'\n 'Staff wear masks' 'Staff get temperature checks'\n 'Staff required to disinfect surfaces between visits'],Health & safety
4,0x88d9050582e1e64b:0x33457ba08c64df20,Service options: ['Online appointments'],Service options
...,...,...,...
82026,0x89b62c2db5856cd3:0x89e0d1a12b77f78a,Accessibility: ['Wheelchair accessible entrance'],Accessibility
82027,0x89b7fdc26ba070ef:0x376e99b8fe4bfc32,Accessibility: ['Wheelchair accessible entrance' 'Wheelchair accessible parking lot'],Accessibility
82028,0x54c0e365e43253f9:0x4366520651be08bb,Accessibility: ['Wheelchair accessible parking lot'],Accessibility
82029,0x54c0e365e43253f9:0x4366520651be08bb,Planning: ['Quick visit'],Planning


In [29]:
# Extraer el contenido después de ':' en la columna 'misc'
misc['misc_content'] = misc['misc'].str.split(':').str[1].str.strip()
# Usar una expresión regular para extraer cada texto dentro de comillas simples
misc['misc_content'] = misc['misc'].str.extractall(r"'(.*?)'").groupby(level=0).agg(list)
# Separar la lista en filas
misc = misc.explode('misc_content')
# Muestra de la tabla
misc = misc[['id_negocio', 'category', 'misc_content']]

In [30]:
misc.rename(columns={'category': 'categoria_atributo'}, inplace=True)
misc.rename(columns={'misc_content': 'atributo'}, inplace=True)
misc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 137923 entries, 0 to 82030
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   id_negocio          137923 non-null  object
 1   categoria_atributo  137923 non-null  object
 2   atributo            137881 non-null  object
dtypes: object(3)
memory usage: 4.2+ MB


In [25]:
# Leer Archivo
misc = pd.read_parquet(r'C:\Users\mauri\OneDrive\Escritorio\proyectogrupal\data\g_misc_v3.parquet', engine='pyarrow')
misc.head()

,id_negocio,categoria_atributo,atributo
0,0x89c2f6163ac2864f:0xa4213a2f66c15d8f,Accessibility,Wheelchair accessible entrance
1,0x864c1f10c079a921:0x95710cd26231cc3,Accessibility,Wheelchair accessible entrance
2,0x88d9050582e1e64b:0x33457ba08c64df20,Service options,Online appointments
3,0x80c2b7bb7415c183:0x3d3b0d5bc46d3b06,Accessibility,Wheelchair accessible entrance
4,0x80c2b7bb7415c183:0x3d3b0d5bc46d3b06,Planning,Quick visit


Conteo de nulos

In [26]:
nulos_por_columna = misc.isnull().sum()
nulos_por_columna

id_negocio            0
categoria_atributo    0
atributo              0
dtype: int64

In [7]:
# Eliminar filas nulas
misc = misc[misc['atributo'].notnull()]
misc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 137881 entries, 0 to 137922
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   id_negocio          137881 non-null  object
 1   categoria_atributo  137881 non-null  object
 2   atributo            137881 non-null  object
dtypes: object(3)
memory usage: 4.2+ MB


Veamos cuantas categorias de atributos hay

In [8]:
categorias = misc['categoria_atributo'].unique()
print(categorias,'\ntotal categorias:',len(categorias))

['Accessibility' 'Health & safety' 'Service options' 'Planning'
 'From the business' 'Amenities' 'Highlights' 'Payments' 'Atmosphere'
 'Crowd' 'Offerings' 'Popular for' 'Dining options' 'Getting here'
 'Recycling' 'Health and safety'] 
total categorias: 16


### Existe una categoria duplicada de distinto nombre: 
- `Health & safety`
- `Health and safety`
La agrupamos en una sola.

In [9]:
misc['categoria_atributo'] = misc['categoria_atributo'].replace({'Health & safety': 'Health and safety'})

veamos si hay relaciones en otras categorias para agruparlas

In [10]:
categorias = misc['categoria_atributo'].unique()

# Iterar a través de cada categoría y mostrar los atributos únicos
for categoria in categorias:
    atributos = misc[misc['categoria_atributo'] == categoria]['atributo'].unique()
    print(f"{categoria}\n", atributos, "\n")

Accessibility
 ['Wheelchair accessible entrance' 'Wheelchair accessible parking lot'
 'Wheelchair accessible restroom' 'Wheelchair accessible elevator'
 'Wheelchair accessible seating' 'Passenger loading area'
 'Assistive hearing loop' 'Wheelchair rental'
 'Wheelchair-accessible entrance' 'Wheelchair-accessible toilet'
 'Wheelchair-accessible car park' 'Wheelchair-accessible seating'
 'Wheelchair-accessible lift'] 

Health and safety
 ['Appointment required' 'Mask required' 'Temperature check required'
 'Staff wear masks' 'Staff get temperature checks'
 'Staff required to disinfect surfaces between visits'
 'Safety dividers at checkout' 'Reservations required'] 

Service options
 ['Online appointments' 'In-store shopping' 'Onsite services' 'Delivery'
 'Curbside pickup' 'In-store pickup' 'Same-day delivery' 'Takeout'
 'Dine-in' 'Online classes' 'Outdoor seating' 'No-contact delivery'
 'Online care' 'Online estimates' 'Drive-through' 'Language assistance'
 'Outdoor services' 'Takeaway' '

### Viendo los atributos comprendemos que Health and Safety es un atributo de la pandemia el cual hoy ya no es necesario ni tenido en cuenta para lugares turisticos. Por ende vamos a eliminar ese atributo.

In [11]:
# Quitamos Health and Safety de los atributos.
misc = misc[misc['categoria_atributo'] != 'Health and safety']
# Comprobamos
categorias = misc['categoria_atributo'].unique()
print(categorias)

['Accessibility' 'Service options' 'Planning' 'From the business'
 'Amenities' 'Highlights' 'Payments' 'Atmosphere' 'Crowd' 'Offerings'
 'Popular for' 'Dining options' 'Getting here' 'Recycling']


### Tambien vemos muchos atributos repetidos. Algunos contienen - y otros solo espacios. Asi que reemplazamos - por espacios y ver de nuevo las categorias

Ademas generalizar todos los tipos de inclusion sexual en una sola categoria. 'LGBTQ Friendly'

In [12]:
# Quitar el guion en los valores de 'atributo'
misc['atributo'] = misc['atributo'].str.replace('-', ' ', regex=False)

# Generalizar el atributo.
mapeo_lgbtq = {
    'LGBTQ friendly': 'LGBTQ friendly',
    'LGBTQ-friendly': 'LGBTQ friendly',
    'Transgender safespace': 'LGBTQ friendly',
    'Transgender safe space': 'LGBTQ friendly'
}

mapeo_accesibilidad = {
    'Wheelchair accessible restroom': 'Wheelchair accessible toilet',
    'Wheelchair accessible parking lot': 'Wheelchair accessible car park',
    'Wheelchair accessible lift': 'Wheelchair accessible elevator'
}

# Aplicar el mapeo a la columna 'atributo'
misc['atributo'] = misc['atributo'].replace(mapeo_lgbtq)
misc['atributo'] = misc['atributo'].replace(mapeo_accesibilidad)


Volvamos a ver los atributos

In [13]:
categorias = misc['categoria_atributo'].unique()

# Iterar a través de cada categoría y mostrar los atributos únicos
for categoria in categorias:
    atributos = misc[misc['categoria_atributo'] == categoria]['atributo'].unique()
    print(f"{categoria}\n", atributos, "\n")

Accessibility
 ['Wheelchair accessible entrance' 'Wheelchair accessible car park'
 'Wheelchair accessible toilet' 'Wheelchair accessible elevator'
 'Wheelchair accessible seating' 'Passenger loading area'
 'Assistive hearing loop' 'Wheelchair rental'] 

Service options
 ['Online appointments' 'In store shopping' 'Onsite services' 'Delivery'
 'Curbside pickup' 'In store pickup' 'Same day delivery' 'Takeout'
 'Dine in' 'Online classes' 'Outdoor seating' 'No contact delivery'
 'Online care' 'Online estimates' 'Drive through' 'Language assistance'
 'Outdoor services' 'Takeaway' 'In store pick up'] 

Planning
 ['Quick visit' 'Dinner reservations recommended' 'Accepts reservations'
 'Appointments recommended' 'Membership required' 'Usually a wait'
 'Brunch reservations recommended' 'Lunch reservations recommended'] 

From the business
 ['Identifies as veteran led' 'Identifies as women led'
 'Identifies as Black owned'] 

Amenities
 ['Gender neutral restroom' 'Wi Fi' 'Good for kids' 'Bar onsi

Vemos que dentro de offering se encuentra un atributo 'braille menu' esto es muy importante para determinar la accesibilidad asi que lo agruparemos en esa caegoria. Ademas el atributo 'High chairs' tambien podria entrar dentro de la accesibilidad para problemas de altura.

In [21]:
# Cambiar la categoría a "Accessibility" para registros con "Offerings" y "Braille menu"
misc.loc[(misc['categoria_atributo'] == 'Offerings') & (misc['atributo'] == 'Braille menu'), 'categoria_atributo'] = 'Accessibility'
misc.loc[(misc['categoria_atributo'] == 'Amenities') & (misc['atributo'] == 'High chairs'), 'categoria_atributo'] = 'Accessibility'
misc.loc[(misc['categoria_atributo'] == 'Accessibility') & (misc['atributo'] == 'Active military discounts'), 'categoria_atributo'] = 'Highlights'


In [23]:
# Muestra final
categorias = misc['categoria_atributo'].unique()

# Iterar a través de cada categoría y mostrar los atributos únicos
for categoria in categorias:
    atributos = misc[misc['categoria_atributo'] == categoria]['atributo'].unique()
    print(f"{categoria}\n", atributos, "\n")

Accessibility
 ['Wheelchair accessible entrance' 'Wheelchair accessible car park'
 'Wheelchair accessible toilet' 'Wheelchair accessible elevator'
 'Wheelchair accessible seating' 'High chairs' 'Passenger loading area'
 'Assistive hearing loop' 'Braille menu' 'Wheelchair rental'] 

Service options
 ['Online appointments' 'In store shopping' 'Onsite services' 'Delivery'
 'Curbside pickup' 'In store pickup' 'Same day delivery' 'Takeout'
 'Dine in' 'Online classes' 'Outdoor seating' 'No contact delivery'
 'Online care' 'Online estimates' 'Drive through' 'Language assistance'
 'Outdoor services' 'Takeaway' 'In store pick up'] 

Planning
 ['Quick visit' 'Dinner reservations recommended' 'Accepts reservations'
 'Appointments recommended' 'Membership required' 'Usually a wait'
 'Brunch reservations recommended' 'Lunch reservations recommended'] 

From the business
 ['Identifies as veteran led' 'Identifies as women led'
 'Identifies as Black owned'] 

Amenities
 ['Gender neutral restroom' 'Wi 

In [24]:
import os
# Guardamos el nuevo Dataframe g_misc
os.makedirs('../../../data', exist_ok=True)
misc.to_parquet('../../../data/g_misc_v3.parquet', index=False)

# Muestra parcial de la tabla y guardado de la informacion.
misc

,id_negocio,categoria_atributo,atributo
0,0x89c2f6163ac2864f:0xa4213a2f66c15d8f,Accessibility,Wheelchair accessible entrance
1,0x864c1f10c079a921:0x95710cd26231cc3,Accessibility,Wheelchair accessible entrance
2,0x8824d759c04cb37b:0xef1cb7398b3d692e,Accessibility,Wheelchair accessible entrance
9,0x88d9050582e1e64b:0x33457ba08c64df20,Service options,Online appointments
10,0x80c2b7bb7415c183:0x3d3b0d5bc46d3b06,Accessibility,Wheelchair accessible entrance
...,...,...,...
137918,0x89b7fdc26ba070ef:0x376e99b8fe4bfc32,Accessibility,Wheelchair accessible car park
137919,0x54c0e365e43253f9:0x4366520651be08bb,Accessibility,Wheelchair accessible car park
137920,0x54c0e365e43253f9:0x4366520651be08bb,Planning,Quick visit
137921,0x54c0e365e43253f9:0x4366520651be08bb,Service options,In store shopping
